In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [2]:
torch.manual_seed(0)

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
class Encoder:
    
    def __init__(self, sentence):
        self.char_list = list(set(sentence))
        self.size = len(self.char_list)
        self.char_dict = {c:i for i, c in enumerate(self.char_list)}
        self.eye = np.eye(self.size)
    
    def int_encode(self, seq):
        return [self.char_dict[c] for c in seq]

    def one_hot_encode(self, seq):
        return self.eye[self.int_encode(seq)]
    
    def decode(self, seq):
        return ''.join([self.char_list[i] for i in seq])

encoder = Encoder(sentence)

In [5]:
sequence_length = 10

x_data = []
y_data = []

for i in range(len(sentence) - sequence_length):
    x_data.append(encoder.one_hot_encode(sentence[i:i + sequence_length]))
    y_data.append(encoder.int_encode(sentence[i+1: i+sequence_length+1]))
    
X = torch.FloatTensor(x_data)
Y = torch.LongTensor(y_data)

In [6]:
class MultiLayerRNN(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, layers):
        super(MultiLayerRNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, X):
        out, _ = self.rnn(X)
        out = self.fc(out)
        return out

<img src="img/rnn_layers.png" width=40% />

In [7]:
input_size = encoder.size
hidden_size = encoder.size
n_layers = 2
n_epochs = 250
learning_rate = 0.1

model = MultiLayerRNN(input_size, hidden_size, n_layers)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [8]:
for epoch in range(1, n_epochs+1):
    
    y_pred = model(X).view(-1, hidden_size)
    loss = criterion(y_pred, Y.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch%50 == 0:
        results = y_pred.argmax(dim=1).tolist()
        predict_str = results[:10] + results[19::10]
        print('---')
        print(f'epoch {epoch:<4}loss:{loss.item():.4f}')
        print('i' + encoder.decode(predict_str))

---
epoch 50  loss:0.4099
ipkyou want to build a ship, don't drum up people together to collect wood and don't dssign them tasks and work, but rather teach them ta long for the endless immensity of the sea.
---
epoch 100 loss:0.2528
it you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
---
epoch 150 loss:0.2587
ig you want to build a ship, don't arum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sea.
---
epoch 200 loss:0.2344
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
---
epoch 250 loss:0.2317
it you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for